In [2]:
from keras.layers import Input,Dense,Conv2D,MaxPooling2D,UpSampling2D,Reshape,Concatenate
from keras.models import Model
from keras import backend as K
from struct import unpack
import json
import glob
import matplotlib.pyplot as plt
from keras.datasets import mnist
from sklearn.model_selection import train_test_split
import numpy as np
from io import BytesIO
import PIL
from PIL import ImageDraw
from keras.utils import get_file
from IPython.display import clear_output,Image,display,HTML

In [3]:
from tensorflow.keras.utils import load_img, img_to_array

In [4]:
labels = ['camel','cat','cow','dog','horse','lion','mouse','pig','rabbit','sheep','tiger','zebra']
labels_to_idx = {label:idx for idx,label in enumerate(labels)}
base_url = 'https://storage.googleapis.com/quickdraw_dataset/full/binary/%s.bin'
paths = {label:get_file(label+'.bin',base_url % label) for label in labels}
paths

25074676/25074676 [==============================] - 1s 0us/step


{'camel': '/home/datalore/.keras/datasets/camel.bin',
 'cat': '/home/datalore/.keras/datasets/cat.bin',
 'cow': '/home/datalore/.keras/datasets/cow.bin',
 'dog': '/home/datalore/.keras/datasets/dog.bin',
 'horse': '/home/datalore/.keras/datasets/horse.bin',
 'lion': '/home/datalore/.keras/datasets/lion.bin',
 'mouse': '/home/datalore/.keras/datasets/mouse.bin',
 'pig': '/home/datalore/.keras/datasets/pig.bin',
 'rabbit': '/home/datalore/.keras/datasets/rabbit.bin',
 'sheep': '/home/datalore/.keras/datasets/sheep.bin',
 'tiger': '/home/datalore/.keras/datasets/tiger.bin',
 'zebra': '/home/datalore/.keras/datasets/zebra.bin'}

In [5]:
def load_icons(paths, train_size=0.85, max_per_class=10000):
    x = []
    y = []
    for label, path in paths.items():
        with open(path, 'rb') as f:
            while True:
                img = PIL.Image.new('L', (32, 32), 'white')
                draw = ImageDraw.Draw(img)
                header = f.read(15)
                if len(header) != 15:
                    break
                strokes, = unpack('H', f.read(2))
                for i in range(strokes):
                    n_points, = unpack('H', f.read(2))
                    fmt = str(n_points) + 'B'
                    read_scaled = lambda: (p // 8 for 
                                           p in unpack(fmt, f.read(n_points)))
                    points = [*zip(read_scaled(), read_scaled())]
                    draw.line(points, fill=0)
                img = img_to_array(img)
                x.append(img)
                y.append(labels_to_idx[label])
                if len(x) % max_per_class == 0:
                    break
    x = np.asarray(x) / 255
    y = np.asarray(y)
    return train_test_split(x, y, train_size=train_size)


x_train, x_test, y_train, y_test = load_icons(paths)
x_train.shape, x_test.shape

((102000, 32, 32, 1), (18000, 32, 32, 1))